In [1]:

import os
from dotenv import load_dotenv
from fireworks.client import Fireworks
import agentops
from agentops.enums import EndState

# Load environment variables
load_dotenv()
FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")

# Initialize AgentOps in development mode
print("Initializing AgentOps in development mode...")
agentops.init(
    api_key=None,  # None for local development
    default_tags=["fireworks-example"]
)
print("Starting new AgentOps session...")
session = agentops.start_session()
print(f"AgentOps initialized. Session URL: {session.session_url}")

# Initialize Fireworks client
print("Initializing Fireworks client...")
client = Fireworks(api_key=FIREWORKS_API_KEY)
print("Fireworks client initialized.")

# Set up messages for story generation
messages = [
    {"role": "system", "content": "You are a creative storyteller."},
    {"role": "user", "content": "Write a short story about a cyber-warrior trapped in the imperial era."}
]


Initializing AgentOps in development mode...


🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=da7c7323-408b-43b9-a503-d7e0a308818a


Starting new AgentOps session...


🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=390c35ff-0df4-4953-9d06-d142fec08719


AgentOps initialized. Session URL: https://app.agentops.ai/drilldown?session_id=390c35ff-0df4-4953-9d06-d142fec08719
Initializing Fireworks client...
Fireworks client initialized.


In [2]:

# Test non-streaming completion
print("Generating story with Fireworks LLM...")
response = client.chat.completions.create(
    model="accounts/fireworks/models/llama-v3p1-8b-instruct",
    messages=messages
)
print("\nLLM Response:")
print(response.choices[0].message.content)
print("\nEvent tracking details:")
print(f"Session URL: {session.session_url}")


Generating story with Fireworks LLM...



LLM Response:
**The Ghost in the Sand**

Akeira crouched behind a crumbling column, her eyes scanning the dusty marketplace for any sign of the enemy. Her sleek, black armor was a stark contrast to the ancient stones and faded awnings of the imperial city. Her breathing was slow and deliberate, a habit honed from years of virtual combat, not the muggy desert air.

"Target offline," her combat comms interrupted, a holographic message flickering to life in front of her. "Ghost-19, priority threat level: Critical."

Akeira's gaze snapped towards the palace at the market's heart, where Emperor Marcus imperiously waved to his subjects from a rooftop balcony. Her finger itched to fire a cyber-warhead at the arrogant leader, but her briefing warned of collateral damage and political fallout. She was forced to play it cool.

"Affirmative, Ghost-19. Scoping primary target... now."

She expertly picked off Imperial Guards one by one, each killing requiring a precise shot, precise control. It wa

In [3]:

# Test streaming completion
print("\nGenerating story with streaming enabled...")
stream = client.chat.completions.create(
    model="accounts/fireworks/models/llama-v3p1-8b-instruct",
    messages=messages,
    stream=True
)

print("\nStreaming LLM Response:")
for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="")
print("\n\nEvent tracking details:")
print(f"Session URL: {session.session_url}")



Generating story with streaming enabled...

Streaming LLM Response:


**The

 Digital Captive**

In the sweltering heat of a Roman sun, Lucius stumbled through the crowded market, his gaze scanning the throngs of pedestrians for any sign of safety. But

 how could he, a cyber warrior, find

 solace in this ancient era? His cybernetic limbs, once tailored for stealth and deception in the vast digital expanse of the net, now felt heavy and cumbersome in his tattered imperial attire.

Just

...

. He yearned to leap back into the digital realm, feeling like a moth trapped in a flame.

With a furtive eye on the shadows, Lucius navigated the crowded streets, dodging glances from astonished civilians. No one suspected a thing

; they assumed him to be just another commoner, bewildered by the vast machinery of the imperial world. As the darkness deepened, Lucius spotted a

 lone figure lurking in the shadows – a young woman with piercing green eyes, clad in a hooded cloak.

"Get word to Zara," the woman

 whispered, as she vanished into the darkness. "Phoenix Rising will come for you. Be ready, Lucien."

Though imprisoned in

 time, Lucien knew his cyber warrior's cry would be heard across the centuries, echoing through the digital expanse, seeking a beacon of hope to guide him back to the digital realm.

Event tracking details:
Session URL: https://app.agentops.ai/drilldown?session_id=390c35ff-0df4-4953-9d06-d142fec08719


In [4]:

# End session and show detailed stats
print("\nEnding AgentOps session...")
try:
    session_stats = session.end_session(
        end_state=EndState.SUCCESS,  # Using the correct enum value
        end_state_reason="Successfully generated stories using both streaming and non-streaming modes."
    )
    print("\nSession Statistics:")
    if session_stats:
        print(f"Total LLM calls: {session_stats.get('llm_calls', 0)}")
        print(f"Total duration: {session_stats.get('duration', 0):.2f}s")
        print(f"Total cost: ${session_stats.get('cost', 0):.4f}")
        print(f"Session URL: {session.session_url}")
    else:
        print("No session statistics available")
except Exception as e:
    print(f"Error ending session: {str(e)}")
    print("Session URL for debugging:", session.session_url)


🖇 AgentOps: Invalid end_state. Please use one of the EndState enums



Ending AgentOps session...



Session Statistics:
No session statistics available
